In [ ]:
# !pip install nltk
# !pip install arabic-reshaper python-bidi
# !pip install langid

In [ ]:
import re
import nltk
import langid
import string
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.isri import ISRIStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances, manhattan_distances, cosine_similarity

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('snowball_data')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package snowball_data to /root/nltk_data...
[nltk_data]   Package snowball_data is already up-to-date!


True

In [ ]:
df = pd.read_excel('/content/Complaints dataset.xlsx')

In [ ]:
df

,OFFER_NAME,CUSTOMER_TYPE,COMPLAINT_TYPE,PRODUCT,CASE_LEVEL_1,CASE_LEVEL_2,CASE_LEVEL_3,CUSTOMER_GROUP,CURRENT_STATUS,ESCALATION_FLAG,...,CLOSE_GROUP,AGE_BRACKET,ACTUAL_COMPLAINT,CALLBACK_MECHANISM,RESOLUTION,RESOLUTION_DESCRIPTION,CASE_DESC,OPEN_GR,Unnamed: 22,Unnamed: 23
0,FTTH Home,CBU,Technical,Internet,Technical complaint,TC_INTERNET_CONFIGURATION,SNW-FTTH - WAN configuration / Tools,FTTH Home,Resolved,No,...,NaN,1,User Not Filled,NaN,NaN,NaN,إعادة برمجة من النظامby Sultan,NaN,NaN,NaN
1,NaN,CBU,Technical,Mobile,Technical complaint,TC_INTERNET_CONNEXION,4G - Frequent Disconnection,NaN,Resolved,Yes,...,Mobile_BO,1,Actual Complaint,SMS,Coverage script was sent to the customer,The customer should pay TF + all due amounts i...,عدم التصفح وسوء الخدمه والرجاء اتخاذ قرار ...,System Administrator,NaN,NaN
2,Super 4G on the go 100GB new,CBU,Commercial,Mobile,Commercial claim,CC_Offers,Embedded Bundle - Didn't Get The Monthly Bundles,Family Mobile-IEW,Resolved,Yes,...,Mobile_BO,1,Actual Complaint,SMS,IEW offer -no link,the line is active now & linked & getting the ...,Activation date Sep 22 2022 لم يتم اضافة ا...,NaN,NaN,NaN
3,Pro Bitstream,CBU,Technical,Internet,Technical complaint,TC_Network,SNW-ADSL-Individual fault,Internet SME,Resolved,No,...,NaN,1,User Not Filled,NaN,NaN,NaN,GAIA,NaN,NaN,NaN
4,FTTH Home,CBU,Technical,Internet,Technical complaint,TC_Network,SNW-FTTH - Down interrupted,FTTH Home,Resolved,No,...,NaN,1,User Not Filled,NaN,NaN,NaN,Interrupted,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276287,Pro BEW 1.2 TB,CBU,Technical,Mobile,Technical complaint,TC_Network,4G+ - No Connection Signal,SME Post-Hybrid,Resolved,No,...,NaN,1,User Not Filled,NaN,NaN,NaN,ئئسسسئ,CBU - CC 1st Layer,NaN,NaN
276288,Super IEW 4G 2Y - 500GB - EOY,CBU,Technical,Mobile,Technical complaint,TC_INTERNET_CONNEXION,4G+ - Slow Download,Internet everywhere IEW,Resolved,No,...,NaN,1,User Not Filled,NaN,NaN,NaN,ئئئ,CBU - CC 1st Layer,NaN,NaN
276289,Super 4G on the go 100GB paid,EBU,Technical,Mobile,Technical complaint,TC_Network,4G+ - No Connection Signal,Business Account,Resolved,No,...,NaN,1,User Not Filled,NaN,NaN,NaN,ئئئء,CBU - CC 1st Layer,NaN,NaN
276290,Prepaid residential Basic,CBU,Commercial,Mobile,Commercial claim,CC_Consumption,Usage Bundles - Data,Prepaid,Resolved,No,...,NaN,1,User Not Filled,NaN,NaN,NaN,ئئئئ,CBU - CC 1st Layer,NaN,NaN


In [ ]:
df['CASE_DESC'].isnull().sum()

0

In [ ]:
df['CASE_DESC'].duplicated().sum()

134789

In [ ]:
df = df.drop_duplicates(subset='CASE_DESC', keep='first')

In [ ]:
df['CASE_DESC'].duplicated().sum()

0

In [ ]:
# def print_arabic_stopwords():
#     arabic_stopwords = set(stopwords.words('arabic'))
#     print("Arabic Stopwords:")
#     for word in arabic_stopwords:
#         print(word)

# # Example usage
# print_arabic_stopwords()

In [ ]:
def preprocess_text(text):
    # 1. Tokenization
    tokens = nltk.word_tokenize(text)

    # 2. Lowercase conversion
    tokens = [token.lower() for token in tokens]

    # 3. Remove punctuation
    tokens = [token for token in tokens if re.match(r'\w', token)]
    tokens = [re.sub("[إأآا]", "ا", token) for token in tokens]
    tokens = [re.sub("ى", "ي", token) for token in tokens]
    tokens = [re.sub("ؤ", "ء", token) for token in tokens]
    tokens = [re.sub("ئ", "ء", token) for token in tokens]
    tokens = [re.sub("ة", "ه", token) for token in tokens]
    tokens = [re.sub("گ", "ك", token) for token in tokens]
    tokens = [token for token in tokens if token not in string.punctuation]


    # 4. Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    stop_words_arabic = set(stopwords.words('arabic'))
    tokens = [token for token in tokens if token not in stop_words_arabic]

    # 5. Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # Initialize the Snowball stemmer for Arabic
    stemmer_arabic = ISRIStemmer()

    # Apply stemming to each token
    tokens = [stemmer_arabic.stem(token) for token in tokens]


    # Join tokens back into text
    processed_text = ' '.join(tokens)

    return processed_text

In [ ]:
df.loc[:, 'processed_text'] = df['CASE_DESC'].apply(lambda x: preprocess_text(x))

<ipython-input-52-3903b24875dc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'processed_text'] = df['CASE_DESC'].apply(lambda x: preprocess_text(x))


In [ ]:
# import langid
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# def most_relevant_complaints(df, employee_query):
#     # Preprocess the query
#     query_lang, _ = langid.classify(employee_query)
#     processed_query = preprocess_text(employee_query, query_lang)

#     # Preprocess the documents
#     preprocessed_documents = df['processed_text'].tolist()
#     processed_documents = [preprocess_text(doc, lang='ar') for doc in preprocessed_documents]

#     vectorizer = TfidfVectorizer()
#     tfidf_matrix = vectorizer.fit_transform([processed_query] + processed_documents)

#     query_tfidf = vectorizer.transform([processed_query])

#     cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix)

#     most_similar_indices = cosine_similarities.argsort()[0][::-1][:5]
#     most_relevant_complaints = [df['processed_text'].iloc[index] for index in most_similar_indices]

#     return most_relevant_complaints

In [ ]:
# def most_relevant_complaints(df, employee_query):
# #     # Preprocess the query

# #     query_lang, _ = langid.classify(employee_query)

#      processed_query = preprocess_text(employee_query)

#      vectorizer = TfidfVectorizer()

#      tfidf_matrix = vectorizer.fit_transform([processed_query] + processed_documents)

#      preprocessed_documents = df['processed_text'].tolist()

# #     # Preprocess the documents
# #     processed_documents = [preprocess_text(doc, lang='ar') for doc in preprocessed_documents]

#      processed_documents = [preprocess_text(doc) for doc in preprocessed_documents]

#      query_tfidf = vectorizer.transform([processed_query])

# #    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix)

# #     most_similar_indices = cosine_similarities.argsort()[0][::-1][:5]
# #     # Calculate cosine similarity between the query and documents
#      cosine_similarities =   cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1]).flatten()
# #     #cosine_similarity(tfidf_matrix[0], tfidf_matrix[1:]).flatten()

# #     # Transform the preprocessed query


# #     # Convert the TF-IDF matrix to a dense array for easier manipulation
# #     #tfidf_matrix = tfidf_matrix.toarray()

# #     # Convert the query TF-IDF to a dense array
# #     # query_tfidf_dense = query_tfidf.toarray()



# #     # Get the indices of the most similar complaints using argpartition
# #     #k = 5  # Number of most similar complaints to retrieve
# #     #most_similar_indices = np.argpartition(cosine_similarities, -k)[-k:][::-1]


# #     # # Get the indices of the most similar complaints
#      most_similar_indices = cosine_similarities.argsort()[:-6:-1]

# #     # Return the most relevant complaints
#      most_relevant_complaints = [df['processed_text'].iloc[index] for index in most_similar_indices]

# #     return most_relevant_complaints

In [ ]:
# def most_relevant_complaints(df, employee_query):
#     # Preprocess the query
#     processed_query = preprocess_text(employee_query)

#     # Get the list of preprocessed documents
#     preprocessed_documents = df['processed_text'].tolist()

#     # Preprocess the documents
#     processed_documents = [preprocess_text(doc) for doc in preprocessed_documents]

#     # Vectorize the documents and the query
#     vectorizer = TfidfVectorizer()
#     tfidf_matrix = vectorizer.fit_transform([processed_query] + processed_documents)

#     # Calculate cosine similarity between the query and documents
#     cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1]).flatten()

#     most_similar_indices = cosine_similarities.argsort()[:-6:-1]

#     # Return the most relevant complaints
#     most_relevant_complaints = [df['processed_text'].iloc[index] for index in most_similar_indices]

#     return most_relevant_complaints

In [ ]:
def most_relevant_complaints(df, employee_query, min_query_length = 2 ):
    # Preprocess the query
    processed_query = preprocess_text(employee_query)

  # Use str.contains with case-insensitive regex to find matching rows
    match_rows = df[df['processed_text'].str.contains(processed_query, case=False)]

    if match_rows.empty:
        print(f"Query '{employee_query}' not found in the document.")
        return []

    # Filter by the minimum query length
    relevant_rows = match_rows[match_rows['processed_text'].apply(lambda x: len(x.split()) > min_query_length)]
    if relevant_rows.empty:
        print(f"No relevant queries '{employee_query}' with more than {min_query_length} words.")
        return []


    # Get the list of preprocessed documents
    preprocessed_documents = df['processed_text'].tolist()

    # Preprocess the documents
    processed_documents = [preprocess_text(doc) for doc in preprocessed_documents]

    all_text = [processed_query] + processed_documents

    # Vectorize the documents and the query
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(all_text)

    query_tfidf = vectorizer.transform([processed_query])


    # Calculate cosine similarity between the query and documents
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()

    most_similar_indices = cosine_similarities.argsort()[:-6:-1]

    # Return the most relevant complaints
    most_relevant_complaints = relevant_rows.iloc[most_similar_indices]['processed_text']
    #[df['processed_text'].iloc[index] for index in most_similar_indices]

    return most_relevant_complaints

In [ ]:
# employee_query = input("Enter Your Query: ")
# result = most_relevant_complaints(df, employee_query)

# # Display the results
# if result:
#     print("Top 5 Most Relevant Complaints:")
#     for i, complaint in enumerate(result, start=1):
#         print(f"{i}. {complaint}")
# else:
#     print("No relevant complaints. Please enter a valid query.")

Enter Your Query: problem


IndexError: ignored

In [ ]:
employee_query = input("Enter Your Query: ")
result = most_relevant_complaints(df, employee_query)

print("Top 5 Most Relevant Complaints:")
for i, complaint in enumerate(result, start=1): # to start the iterate from index number 1
    print(f"{i}. {complaint}")

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from arabic_reshaper import reshape
from bidi.algorithm import get_display


result = most_relevant_complaints(df, employee_query)

G = nx.DiGraph()

# Add nodes and edges based on transitions
for i in range(len(result) -1):
    source_complaint = result[i]
    target_complaint = result[i + 1]
    G.add_edge(source_complaint, target_complaint)

    self_loops = [(node, node) for node in G.nodes if G.has_edge(node, node)]
G.remove_edges_from(self_loops)

pos = nx.spring_layout(G)  # Set layout for better visualization
reshaped_labels = {node: get_display(reshape(node)) for node in G.nodes()}
nx.draw(G, pos, with_labels=True, font_weight='bold', node_color='skyblue', labels=reshaped_labels, node_size=2000, font_size=9, font_color='black', edge_color='black')
plt.show()